# 03 - Data Augmentation Gallery

**Course:** 21CSE558T - Deep Neural Network Architectures  
**Module 4:** CNNs (Week 2 of 3)  
**Date:** October 31, 2025  
**Duration:** 45-60 minutes

---

## Learning Objectives

By the end of this notebook, you will be able to:
1. Apply geometric and photometric augmentation techniques
2. Understand domain-appropriate augmentation selection
3. Visualize augmentation effects on real images
4. Create custom augmentation pipelines
5. Compare training with/without augmentation

---

## Story: Character: Arun's Photography Studio

**Character: Arun** runs a wildlife photography training school. He teaches students to recognize animals from photos taken in various conditions:
- Different angles (rotation)
- Different distances (zoom)
- Different lighting (brightness)
- Different weather (contrast, color)

Character: Arun discovered that students trained on **diverse photos** perform better in real safaris than those trained on **perfect studio photos only**.

**This is exactly what data augmentation does for CNNs!**

---

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, BatchNormalization, Activation,
    MaxPooling2D, GlobalAveragePooling2D,
    Dropout, Dense
)
import warnings
warnings.filterwarnings('ignore')

print(f"TensorFlow version: {tf.__version__}")
print(f"NumPy version: {np.__version__}")

## Part 1: Geometric Augmentation Gallery

**Geometric transformations** change the spatial properties of images:
- Rotation
- Horizontal/Vertical Flip
- Width/Height Shift
- Zoom
- Shear

### Analogy: Character: Priya Takes Photos of Her Dog

**Character: Priya** wants to train a model to recognize her dog. She takes photos from:
- Different angles → **Rotation augmentation**
- Left and right sides → **Horizontal flip**
- Near and far → **Zoom augmentation**
- While the dog moves → **Shift augmentation**

This creates a **robust** model that recognizes the dog regardless of position!

In [ ]:
# Load a sample image from CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# CIFAR-10 class names
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
               'dog', 'frog', 'horse', 'ship', 'truck']

# Select a sample image (let's pick a dog)
sample_idx = np.where(y_train == 5)[0][0]  # Class 5 = dog
sample_image = x_train[sample_idx]
sample_label = class_names[y_train[sample_idx][0]]

print(f"Sample image: {sample_label}")
print(f"Image shape: {sample_image.shape}")

plt.figure(figsize=(4, 4))
plt.imshow(sample_image)
plt.title(f"Original: {sample_label}")
plt.axis('off')
plt.show()

In [ ]:
# Geometric Augmentation Gallery
def show_geometric_augmentations(image):
    """
    Display various geometric augmentations.
    
    Character: Arun's different camera angles!
    """
    # Prepare image for generator (needs batch dimension)
    img_expanded = np.expand_dims(image, axis=0)
    
    # Define different augmentation strategies
    augmentations = {
        'Rotation (±20°)': ImageDataGenerator(rotation_range=20),
        'Horizontal Flip': ImageDataGenerator(horizontal_flip=True),
        'Vertical Flip': ImageDataGenerator(vertical_flip=True),
        'Width Shift (±20%)': ImageDataGenerator(width_shift_range=0.2),
        'Height Shift (±20%)': ImageDataGenerator(height_shift_range=0.2),
        'Zoom (80-120%)': ImageDataGenerator(zoom_range=0.2),
        'Shear (20°)': ImageDataGenerator(shear_range=20),
        'Combined': ImageDataGenerator(
            rotation_range=15,
            width_shift_range=0.1,
            height_shift_range=0.1,
            horizontal_flip=True,
            zoom_range=0.1
        )
    }
    
    # Create gallery
    fig, axes = plt.subplots(3, 3, figsize=(12, 12))
    axes = axes.flatten()
    
    # Show original
    axes[0].imshow(image.astype('uint8'))
    axes[0].set_title('Original', fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    # Show augmentations
    for idx, (name, datagen) in enumerate(augmentations.items(), 1):
        # Generate one augmented image
        aug_iter = datagen.flow(img_expanded, batch_size=1)
        aug_image = next(aug_iter)[0].astype('uint8')
        
        axes[idx].imshow(aug_image)
        axes[idx].set_title(name, fontsize=12)
        axes[idx].axis('off')
    
    plt.suptitle("Geometric Augmentation Gallery\nCharacter: Arun's Different Camera Angles", 
                 fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()

show_geometric_augmentations(sample_image)

## Part 2: Photometric Augmentation Gallery

**Photometric transformations** change the pixel intensities:
- Brightness adjustment
- Contrast adjustment
- Saturation adjustment
- Hue shift
- Channel shift

### Analogy: Character: Lakshmi's Weather Conditions

**Character: Lakshmi** is a drone photographer who captures landscapes in:
- Bright sunlight → **High brightness**
- Cloudy weather → **Low brightness**
- Morning golden hour → **Warm colors**
- Evening blue hour → **Cool colors**

Models trained on **diverse lighting conditions** work better in real-world scenarios!

In [ ]:
# Photometric Augmentation Gallery
def show_photometric_augmentations(image):
    """
    Display various photometric augmentations.
    
    Character: Lakshmi's different weather/lighting conditions!
    """
    # Prepare image
    img_expanded = np.expand_dims(image, axis=0)
    
    # Define photometric augmentations
    augmentations = {
        'Brightness (+30%)': ImageDataGenerator(brightness_range=[1.0, 1.3]),
        'Brightness (-30%)': ImageDataGenerator(brightness_range=[0.7, 1.0]),
        'Channel Shift (R)': ImageDataGenerator(channel_shift_range=50.0),
        'Combined Photometric': ImageDataGenerator(
            brightness_range=[0.8, 1.2],
            channel_shift_range=30.0
        )
    }
    
    # Additional manual augmentations
    def adjust_contrast(img, factor):
        """Adjust image contrast."""
        mean = np.mean(img)
        return np.clip((img - mean) * factor + mean, 0, 255).astype('uint8')
    
    def adjust_saturation(img, factor):
        """Adjust image saturation."""
        hsv = tf.image.rgb_to_hsv(img / 255.0)
        hsv = tf.concat([
            hsv[:, :, 0:1],
            hsv[:, :, 1:2] * factor,
            hsv[:, :, 2:3]
        ], axis=-1)
        rgb = tf.image.hsv_to_rgb(hsv)
        return (rgb.numpy() * 255).astype('uint8')
    
    # Create gallery
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.flatten()
    
    # Show original
    axes[0].imshow(image.astype('uint8'))
    axes[0].set_title('Original', fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    # Show ImageDataGenerator augmentations
    for idx, (name, datagen) in enumerate(augmentations.items(), 1):
        aug_iter = datagen.flow(img_expanded, batch_size=1)
        aug_image = next(aug_iter)[0].astype('uint8')
        
        axes[idx].imshow(aug_image)
        axes[idx].set_title(name, fontsize=12)
        axes[idx].axis('off')
    
    # Show manual augmentations
    axes[5].imshow(adjust_contrast(image.astype('float32'), 1.5))
    axes[5].set_title('Contrast (+50%)', fontsize=12)
    axes[5].axis('off')
    
    axes[6].imshow(adjust_contrast(image.astype('float32'), 0.5))
    axes[6].set_title('Contrast (-50%)', fontsize=12)
    axes[6].axis('off')
    
    axes[7].imshow(adjust_saturation(image.astype('float32'), 1.8))
    axes[7].set_title('Saturation (+80%)', fontsize=12)
    axes[7].axis('off')
    
    plt.suptitle("Photometric Augmentation Gallery\nCharacter: Lakshmi's Different Lighting Conditions", 
                 fontsize=16, fontweight='bold', y=0.98)
    plt.tight_layout()
    plt.show()

show_photometric_augmentations(sample_image)

## Part 3: Domain-Appropriate Augmentation

**CRITICAL CONCEPT:** Not all augmentations are appropriate for all domains!

### Examples:

| Domain | Good Augmentations ✅ | Bad Augmentations ❌ |
|--------|----------------------|----------------------|
| **Cats/Dogs** | Rotation ±15°, Flip horizontal, Zoom, Brightness | Vertical flip, Rotation ±180° |
| **Handwritten Digits** | Rotation ±10°, Shift, Zoom | Flips (6→9 problem!), Heavy rotation |
| **Medical X-rays** | Shift, Zoom, Brightness | Flips (left≠right organs!), Heavy rotation |
| **Satellite Images** | All rotations, All flips | (Almost everything works) |
| **Road Signs** | Slight rotation ±5°, Brightness | Flips (make invalid signs!), Heavy rotation |

### Story: Character: Vikram's Mistake

**Character: Vikram** trained a digit recognizer with vertical flips. His model learned:
- 6 = 9 (upside down)
- b = q (flipped)
- d = p (flipped)

**Test accuracy dropped from 95% to 60%!**

**Lesson:** Always consider domain constraints before augmentation!

In [ ]:
# Compare domain-appropriate vs inappropriate augmentation
def compare_augmentation_strategies(image, label):
    """
    Compare appropriate vs inappropriate augmentation.
    
    Character: Vikram learns from his mistakes!
    """
    img_expanded = np.expand_dims(image, axis=0)
    
    # Appropriate for natural images (cats, dogs, etc.)
    appropriate_datagen = ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        zoom_range=0.1,
        brightness_range=[0.8, 1.2]
    )
    
    # Inappropriate (too aggressive)
    inappropriate_datagen = ImageDataGenerator(
        rotation_range=180,  # Too much!
        width_shift_range=0.5,  # Too much!
        height_shift_range=0.5,  # Too much!
        horizontal_flip=True,
        vertical_flip=True,  # Dogs don't walk upside down!
        zoom_range=0.8,  # Too much!
        brightness_range=[0.2, 2.0]  # Too extreme!
    )
    
    # Generate samples
    fig, axes = plt.subplots(2, 5, figsize=(18, 8))
    
    # Appropriate augmentations
    axes[0, 0].imshow(image.astype('uint8'))
    axes[0, 0].set_title('Original', fontsize=12, fontweight='bold')
    axes[0, 0].axis('off')
    
    app_iter = appropriate_datagen.flow(img_expanded, batch_size=1)
    for i in range(1, 5):
        aug_image = next(app_iter)[0].astype('uint8')
        axes[0, i].imshow(aug_image)
        axes[0, i].set_title(f'✅ Appropriate #{i}', fontsize=12, color='green')
        axes[0, i].axis('off')
    
    # Inappropriate augmentations
    axes[1, 0].imshow(image.astype('uint8'))
    axes[1, 0].set_title('Original', fontsize=12, fontweight='bold')
    axes[1, 0].axis('off')
    
    inapp_iter = inappropriate_datagen.flow(img_expanded, batch_size=1)
    for i in range(1, 5):
        aug_image = next(inapp_iter)[0].astype('uint8')
        axes[1, i].imshow(aug_image)
        axes[1, i].set_title(f'❌ Too Aggressive #{i}', fontsize=12, color='red')
        axes[1, i].axis('off')
    
    plt.suptitle(f"Domain-Appropriate Augmentation Comparison: {label}\nCharacter: Vikram's Lesson", 
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

compare_augmentation_strategies(sample_image, sample_label)

## Part 4: Training Comparison - With vs Without Augmentation

**The ultimate test:** Does augmentation actually improve performance?

Let's train two small CNNs on CIFAR-10:
1. **Without augmentation** (baseline)
2. **With augmentation** (improved)

**Prediction:**
- Without: High train acc (95%), Low test acc (65%) → **Overfitting**
- With: Medium train acc (85%), High test acc (82%) → **Generalization**

### Story: Character: Ananya's Two Students

**Character: Ananya** is a driving instructor with two students:
- **Student Rohan:** Practices only on sunny days, perfect roads
- **Student Meera:** Practices in rain, night, traffic, different roads

**Driving test day (rainy!):**
- Rohan: Nervous, makes mistakes (overfitted to perfect conditions)
- Meera: Confident, passes easily (generalized to all conditions)

**This is data augmentation!**

In [ ]:
# Prepare small dataset for quick training
# Use only 5000 training samples for faster demonstration
(x_train_full, y_train_full), (x_test_full, y_test_full) = cifar10.load_data()

# Take subset
x_train_small = x_train_full[:5000].astype('float32') / 255.0
y_train_small = tf.keras.utils.to_categorical(y_train_full[:5000], 10)
x_test_small = x_test_full[:1000].astype('float32') / 255.0
y_test_small = tf.keras.utils.to_categorical(y_test_full[:1000], 10)

print(f"Training samples: {x_train_small.shape[0]}")
print(f"Test samples: {x_test_small.shape[0]}")
print("\nNote: Using small dataset for quick demonstration!")

In [ ]:
# Define a simple CNN model
def create_cnn():
    """
    Create a simple modern CNN.
    
    Following Week 11 best practices:
    - Conv → BN → ReLU pattern
    - Global Average Pooling
    - Dropout for regularization
    """
    model = Sequential([
        # Block 1: 32 filters
        Conv2D(32, (3,3), padding='same', input_shape=(32, 32, 3)),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(32, (3,3), padding='same'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D((2,2)),
        Dropout(0.2),
        
        # Block 2: 64 filters
        Conv2D(64, (3,3), padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(64, (3,3), padding='same'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D((2,2)),
        Dropout(0.3),
        
        # Block 3: 128 filters
        Conv2D(128, (3,3), padding='same'),
        BatchNormalization(),
        Activation('relu'),
        GlobalAveragePooling2D(),
        
        # Output
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Create both models
model_no_aug = create_cnn()
model_with_aug = create_cnn()

print("Model architecture (same for both):")
model_no_aug.summary()

In [ ]:
# Train Model 1: WITHOUT augmentation
print("=" * 60)
print("Training Model 1: WITHOUT Data Augmentation")
print("(Character: Rohan - practices only on sunny days)")
print("=" * 60)

history_no_aug = model_no_aug.fit(
    x_train_small, y_train_small,
    validation_data=(x_test_small, y_test_small),
    epochs=20,
    batch_size=64,
    verbose=1
)

print("\n✅ Training without augmentation complete!")

In [ ]:
# Train Model 2: WITH augmentation
print("=" * 60)
print("Training Model 2: WITH Data Augmentation")
print("(Character: Meera - practices in all conditions)")
print("=" * 60)

# Create augmentation generator
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1
)

# Flow training data
train_generator = train_datagen.flow(
    x_train_small, y_train_small,
    batch_size=64
)

history_with_aug = model_with_aug.fit(
    train_generator,
    validation_data=(x_test_small, y_test_small),
    epochs=20,
    steps_per_epoch=len(x_train_small) // 64,
    verbose=1
)

print("\n✅ Training with augmentation complete!")

In [ ]:
# Compare training curves
def plot_comparison(history1, history2):
    """
    Compare training histories.
    
    Character: Ananya compares her two students' progress!
    """
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Accuracy comparison
    axes[0].plot(history1.history['accuracy'], 'b-', label='Train (No Aug)', linewidth=2)
    axes[0].plot(history1.history['val_accuracy'], 'b--', label='Val (No Aug)', linewidth=2)
    axes[0].plot(history2.history['accuracy'], 'g-', label='Train (With Aug)', linewidth=2)
    axes[0].plot(history2.history['val_accuracy'], 'g--', label='Val (With Aug)', linewidth=2)
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Accuracy', fontsize=12)
    axes[0].set_title('Accuracy Comparison', fontsize=14, fontweight='bold')
    axes[0].legend(fontsize=11)
    axes[0].grid(True, alpha=0.3)
    
    # Loss comparison
    axes[1].plot(history1.history['loss'], 'b-', label='Train (No Aug)', linewidth=2)
    axes[1].plot(history1.history['val_loss'], 'b--', label='Val (No Aug)', linewidth=2)
    axes[1].plot(history2.history['loss'], 'g-', label='Train (With Aug)', linewidth=2)
    axes[1].plot(history2.history['val_loss'], 'g--', label='Val (With Aug)', linewidth=2)
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('Loss', fontsize=12)
    axes[1].set_title('Loss Comparison', fontsize=14, fontweight='bold')
    axes[1].legend(fontsize=11)
    axes[1].grid(True, alpha=0.3)
    
    plt.suptitle("Training Comparison: With vs Without Augmentation\nCharacter: Ananya's Students", 
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

plot_comparison(history_no_aug, history_with_aug)

In [ ]:
# Final results comparison
print("=" * 70)
print("FINAL RESULTS COMPARISON")
print("=" * 70)

# Get final epoch results
no_aug_train_acc = history_no_aug.history['accuracy'][-1]
no_aug_val_acc = history_no_aug.history['val_accuracy'][-1]
no_aug_gap = no_aug_train_acc - no_aug_val_acc

with_aug_train_acc = history_with_aug.history['accuracy'][-1]
with_aug_val_acc = history_with_aug.history['val_accuracy'][-1]
with_aug_gap = with_aug_train_acc - with_aug_val_acc

print("\n📊 WITHOUT Data Augmentation (Character: Rohan):")
print(f"   Train Accuracy: {no_aug_train_acc:.1%}")
print(f"   Val Accuracy:   {no_aug_val_acc:.1%}")
print(f"   Gap:            {no_aug_gap:.1%} ⚠️  (Overfitting!)")

print("\n📊 WITH Data Augmentation (Character: Meera):")
print(f"   Train Accuracy: {with_aug_train_acc:.1%}")
print(f"   Val Accuracy:   {with_aug_val_acc:.1%}")
print(f"   Gap:            {with_aug_gap:.1%} ✅ (Good generalization!)")

print("\n🎯 IMPROVEMENT:")
print(f"   Val Accuracy:   +{(with_aug_val_acc - no_aug_val_acc):.1%}")
print(f"   Overfitting:    {(no_aug_gap / with_aug_gap):.1f}× less")

print("\n💡 KEY INSIGHT:")
print("   Data augmentation trades a bit of training accuracy")
print("   for MUCH BETTER generalization to unseen data!")
print("="*70)

## Part 5: Custom Augmentation Pipeline

**Advanced Topic:** Creating custom augmentation pipelines for specific domains.

### Example: Wildlife Camera Trap Images

**Character: Rohini** analyzes camera trap images. Her requirements:
1. Animals appear at any angle → **Rotation ±30°**
2. Day and night captures → **Brightness 50-150%**
3. Animals may be far or near → **Zoom 80-120%**
4. Camera on left or right of trail → **Horizontal flip**
5. But: Animals always right-side up → **NO vertical flip**
6. But: Camera fixed position → **Minimal shift**

In [ ]:
# Custom augmentation pipeline for wildlife
wildlife_augmentation = ImageDataGenerator(
    rotation_range=30,           # Animals at any angle
    width_shift_range=0.05,      # Minimal shift (fixed camera)
    height_shift_range=0.05,     # Minimal shift (fixed camera)
    horizontal_flip=True,        # Left or right of trail
    vertical_flip=False,         # Animals don't walk upside down!
    zoom_range=0.2,              # Far or near
    brightness_range=[0.5, 1.5], # Day and night
    fill_mode='constant',        # Black padding
    cval=0                       # Black color
)

print("Custom Wildlife Augmentation Pipeline:")
print("Character: Rohini's domain-specific requirements\n")

# Show examples
img_expanded = np.expand_dims(sample_image, axis=0)
wildlife_iter = wildlife_augmentation.flow(img_expanded, batch_size=1)

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

axes[0].imshow(sample_image.astype('uint8'))
axes[0].set_title('Original', fontsize=12, fontweight='bold')
axes[0].axis('off')

for i in range(1, 8):
    aug_image = next(wildlife_iter)[0].astype('uint8')
    axes[i].imshow(aug_image)
    axes[i].set_title(f'Wildlife Aug #{i}', fontsize=12)
    axes[i].axis('off')

plt.suptitle("Custom Wildlife Augmentation Pipeline\nCharacter: Rohini's Domain-Specific Strategy", 
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## Part 6: Practice Exercises

**Apply what you learned!**

In [ ]:
print("PRACTICE EXERCISES")
print("=" * 70)
print()
print("Exercise 1: Medical X-ray Augmentation")
print("-" * 70)
print("Design an augmentation pipeline for chest X-rays.")
print("Constraints:")
print("  - Left lung ≠ Right lung (NO horizontal flip!)")
print("  - Patients are upright (NO vertical flip!)")
print("  - Slight positioning variations OK")
print("  - Different X-ray machine intensities")
print()
print("TODO: Complete the code below:")
print()

# YOUR CODE HERE
medical_augmentation = ImageDataGenerator(
    # Add your augmentation parameters
)

print()
print("Exercise 2: Traffic Sign Recognition")
print("-" * 70)
print("Design augmentation for traffic sign classification.")
print("Constraints:")
print("  - Signs must remain readable")
print("  - Slight angles OK (camera not perfectly aligned)")
print("  - Different weather lighting")
print("  - NO flips (would create invalid signs!)")
print()
print("TODO: Complete the code below:")
print()

# YOUR CODE HERE
traffic_sign_augmentation = ImageDataGenerator(
    # Add your augmentation parameters
)

print()
print("Exercise 3: Satellite Image Classification")
print("-" * 70)
print("Design augmentation for land use classification from satellites.")
print("Constraints:")
print("  - All rotations valid (no 'up' direction in space)")
print("  - All flips valid")
print("  - Seasonal color variations")
print("  - Cloud shadows (brightness variation)")
print()
print("TODO: Complete the code below:")
print()

# YOUR CODE HERE
satellite_augmentation = ImageDataGenerator(
    # Add your augmentation parameters
)

print()
print("=" * 70)

## Summary: Key Takeaways

### 1. What is Data Augmentation?
**Creating synthetic training variations** to improve generalization.

### 2. Types of Augmentation
- **Geometric:** Rotation, flip, shift, zoom, shear
- **Photometric:** Brightness, contrast, saturation, hue

### 3. Domain Appropriateness is CRITICAL
- Cats/Dogs: Most augmentations OK
- Digits: NO flips (6→9 problem)
- Medical: NO flips (anatomy matters)
- Satellites: ALL augmentations OK

### 4. Training Benefits
- Reduces overfitting
- Improves test accuracy
- Creates robust models
- Trades train accuracy for generalization

### 5. Implementation in Keras
```python
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1,
    brightness_range=[0.8, 1.2]
)
```

### 6. Golden Rules
- ✅ DO augment training data
- ❌ DON'T augment validation/test data
- ✅ DO consider domain constraints
- ❌ DON'T use all augmentations blindly
- ✅ DO experiment and validate

### 7. Character Lessons
- **Character: Arun:** Diverse training beats perfect studio shots
- **Character: Lakshmi:** Weather variations create robustness
- **Character: Vikram:** Domain constraints matter (6≠9!)
- **Character: Ananya:** Practice in varied conditions = generalization
- **Character: Rohini:** Custom pipelines for specific domains

---

## Next Steps

1. Complete practice exercises above
2. Read quick reference cheat sheet
3. Complete architecture design worksheet
4. Review Notebook 04 (regularization comparison)
5. Prepare for Tutorial T11 (Monday, Nov 3)

---

**🎯 Learning Objective Check:**
- ✅ Can apply geometric and photometric augmentation
- ✅ Understand domain-appropriate selection
- ✅ Visualized augmentation effects
- ✅ Created custom augmentation pipelines
- ✅ Compared training with/without augmentation

**Congratulations! You've mastered data augmentation techniques!** 🎉